# **Final Project Task 5 - Census Modeling NN Regression**


Requirements

- Create a NN regression model on the Census dataset, with 'hours-per-week' target

- Model Selection and Setup:
    - Build a neural network model using a deep learning library like TensorFlow, Keras or PyTorch.
    - Choose a loss (or experiment with different losses) for the model and justify the choice.
        - MSE, MAE, RMSE, Huber Loss or others
    - Justify model choices based on dataset characteristics and task requirements; specify model pros and cons.


- Data Preparation
    - Use the preprocessed datasets from Task 1.
    - From the train set, create an extra validation set, if necesarry. So in total there will be: train, validation and test datasets.
    - Be sure all models have their data preprocessed as needed. Some models require different, or no encoding for some features.


- Model Training and Experimentation
    - Establish a Baseline Model:
        - Train a simple NN model with default settings as a baseline.
        - Evaluate its performance to establish a benchmark for comparison.
    - Make plots with train, validation loss and metric on epochs (or on steps), if applicable.
    - Feature Selection:
        - Neural Networks can learn feature importance automatically, so all relevant features should be included rather than manually selecting a subset.
        - Consider using embeddings for high-cardinality categorical features instead of one-hot encoding to improve efficiency.
    - Experimentation:
        - Focus on preprocessing techniques rather than manually selecting feature combinations. Ensure numerical features are normalized (e.g., MinMaxScaler, StandardScaler) and categorical features are properly encoded (e.g., one-hot encoding or embeddings for high-cardinality variables).
        - Experiment with different neural network architectures (e.g., number of layers, neurons per layer) and hyperparameters (e.g., activation functions, learning rates, dropout rates, and batch sizes).
        - Use techniques such as early stopping and learning rate scheduling to optimize model performance and prevent overfitting.
        - Identify the best model which have the best performance metrics on test set.
    - Hyperparameter Tuning:
        - Perform hyperparameter tuning only on the best-performing model after evaluating all model types and experiments.
        - Consider using techniques like Grid Search for exhaustive tuning, Random Search for quicker exploration, or Bayesian Optimization for an intelligent, efficient search of hyperparameters.
        - Avoid tuning models that do not show strong baseline performance or are unlikely to outperform others based on experimentation.
        - Ensure that hyperparameter tuning is done after completing feature selection, baseline modeling, and experimentation, ensuring that the model is stable and representative of the dataset.


- Model Evaluation
    - Evaluate models on the test dataset using regression metrics:
        - Mean Absolute Error (MAE)
        - Mean Squared Error (MSE)
        - Root Mean Squared Error (RMSE)
        - R² Score
    - Choose one metric for model comparison and explain your choice
    - Compare the results across different models. Save all experiment results into a table.



Deliverables

- Notebook code with no errors.
- Code and results from experiments. Create a table with all experiments results, include experiment name, metrics results.
- Explain findings, choices, results.
- Potential areas for improvement or further exploration.


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.neural_network import MLPRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


1. Încărcarea datelor preprocesate (Task 1)

In [2]:
import pandas as pd

X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")

y_train = pd.read_csv("y_train.csv").values.ravel()
y_test = pd.read_csv("y_test.csv").values.ravel()


În acest task sunt utilizate datele preprocesate obținute în Task 1.
Seturile X_train, X_test, y_train și y_test conțin deja variabilele numerice scalate și variabilele categoriale codificate.
Astfel, nu mai este necesară refacerea etapelor de curățare și preprocesare a datelor.
Funcția values.ravel() este utilizată pentru a transforma vectorul țintă într-un format compatibil cu MLPRegressor.

2. Creez set de VALIDARE

In [3]:
from sklearn.model_selection import train_test_split

X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)


În această etapă, setul de antrenare obținut în Task 1 a fost împărțit suplimentar pentru a crea un set de validare. Setul de validare este utilizat pentru evaluarea intermediară a modelului și pentru a observa comportamentul acestuia în timpul experimentelor, fără a afecta setul de test. Împărțirea a fost realizată folosind o proporție de 80% pentru antrenare și 20% pentru validare, cu un `random_state` fix pentru reproductibilitatea rezultatelor.


3. Baseline Neural Network

In [4]:
from sklearn.neural_network import MLPRegressor

baseline_nn = MLPRegressor(
    hidden_layer_sizes=(64,),
    activation="relu",
    solver="adam",
    max_iter=200,
    random_state=42
)

baseline_nn.fit(X_train_final, y_train_final)


c:\Users\Doina\Desktop\ADC\AN 2\5. Machine Learning\ML_Lectures_datele initiale\.venv311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:785: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


,"loss loss: {'squared_error', 'poisson'}, default='squared_error'The loss function to use when training the weights. Note that the""squared error"" and ""poisson"" losses actually implement""half squares error"" and ""half poisson deviance"" to simplify thecomputation of the gradient. Furthermore, the ""poisson"" loss internally usesa log-link (exponential as the output activation function) and requires``y >= 0``... versionchanged:: 1.7 Added parameter `loss` and option 'poisson'.",'squared_error'
,"hidden_layer_sizes hidden_layer_sizes: array-like of shape(n_layers - 2,), default=(100,)The ith element represents the number of neurons in the ithhidden layer.","(64,)"
,"activation activation: {'identity', 'logistic', 'tanh', 'relu'}, default='relu'Activation function for the hidden layer.- 'identity', no-op activation, useful to implement linear bottleneck, returns f(x) = x- 'logistic', the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).- 'tanh', the hyperbolic tan function, returns f(x) = tanh(x).- 'relu', the rectified linear unit function, returns f(x) = max(0, x)",'relu'
,"solver solver: {'lbfgs', 'sgd', 'adam'}, default='adam'The solver for weight optimization.- 'lbfgs' is an optimizer in the family of quasi-Newton methods.- 'sgd' refers to stochastic gradient descent.- 'adam' refers to a stochastic gradient-based optimizer proposed by Kingma, Diederik, and Jimmy BaFor a comparison between Adam optimizer and SGD, see:ref:`sphx_glr_auto_examples_neural_networks_plot_mlp_training_curves.py`.Note: The default solver 'adam' works pretty well on relativelylarge datasets (with thousands of training samples or more) in terms ofboth training time and validation score.For small datasets, however, 'lbfgs' can converge faster and performbetter.",'adam'
,"alpha alpha: float, default=0.0001Strength of the L2 regularization term. The L2 regularization termis divided by the sample size when added to the loss.",0.0001
,"batch_size batch_size: int, default='auto'Size of minibatches for stochastic optimizers.If the solver is 'lbfgs', the regressor will not use minibatch.When set to ""auto"", `batch_size=min(200, n_samples)`.",'auto'
,"learning_rate learning_rate: {'constant', 'invscaling', 'adaptive'}, default='constant'Learning rate schedule for weight updates.- 'constant' is a constant learning rate given by 'learning_rate_init'.- 'invscaling' gradually decreases the learning rate ``learning_rate_`` at each time step 't' using an inverse scaling exponent of 'power_t'. effective_learning_rate = learning_rate_init / pow(t, power_t)- 'adaptive' keeps the learning rate constant to 'learning_rate_init' as long as training loss keeps decreasing. Each time two consecutive epochs fail to decrease training loss by at least tol, or fail to increase validation score by at least tol if 'early_stopping' is on, the current learning rate is divided by 5.Only used when solver='sgd'.",'constant'
,"learning_rate_init learning_rate_init: float, default=0.001The initial learning rate used. It controls the step-sizein updating the weights. Only used when solver='sgd' or 'adam'.",0.001
,"power_t power_t: float, default=0.5The exponent for inverse scaling learning rate.It is used in updating effective learning rate when the learning_rateis set to 'invscaling'. Only used when solver='sgd'.",0.5
,"max_iter max_iter: int, default=200Maximum number of iterations. The solver iterates until convergence(determined by 'tol') or this number of iterations. For stochasticsolvers ('sgd', 'adam'), note that this determines the number of epochs(how many times each data point will be used), not the number ofgradient steps.",200
,"shuffle shuffle: bool, default=TrueWhether to shuffle samples in each iteration. Only used whensolver='sgd' or 'adam'.",True


A fost construit un model de tip Neural Network de bază (baseline) folosind `MLPRegressor`, cu un singur strat ascuns format din 64 de neuroni. Acest model are rolul de punct de referință pentru comparația cu arhitecturi mai complexe. Parametrii utilizați, precum funcția de activare ReLU și optimizatorul Adam, reprezintă alegeri standard pentru probleme de regresie și permit o antrenare stabilă a rețelei. Numărul de iterații a fost limitat pentru a menține simplitatea modelului de bază.


4. Evaluare Baseline

In [5]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

y_pred = baseline_nn.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

mae, mse, rmse, r2


(7.630589892064927,
 119.25750160784743,
 np.float64(10.920508303547386),
 0.22033204170340326)

MAE  = 7.63 În medie, predicțiile modelului diferă de valoarea reală cu aprox. 7–8 ore pe săptămână. Modelul greșește, în medie, cu mai puțin de o zi de muncă.

MSE  = 119.25 Erorile mari sunt penalizate mai puternic (pătrat).

RMSE = 10.92 Metrică principală. Când apar erori mari, modelul poate greși cu ~11 ore/săptămână.

R²   = 0.220 Modelul explică aproximativ 24% din variația numărului de ore lucrate pe săptămână.R² > 0 → modelul este mai bun decât o predicție medie constantă.


Baseline Neural Network 

oferă o performanță modestă, dar validă

stabilește un punct de referință

lasă loc clar de îmbunătățiri

5. Model NN mai complex (experiment)

In [7]:
nn_v2 = MLPRegressor(
    hidden_layer_sizes=(128, 64),
    activation="relu",
    solver="adam",
    max_iter=100,
    random_state=42
)

nn_v2.fit(X_train_final, y_train_final)


c:\Users\Doina\Desktop\ADC\AN 2\5. Machine Learning\ML_Lectures_datele initiale\.venv311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:785: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


,"loss loss: {'squared_error', 'poisson'}, default='squared_error'The loss function to use when training the weights. Note that the""squared error"" and ""poisson"" losses actually implement""half squares error"" and ""half poisson deviance"" to simplify thecomputation of the gradient. Furthermore, the ""poisson"" loss internally usesa log-link (exponential as the output activation function) and requires``y >= 0``... versionchanged:: 1.7 Added parameter `loss` and option 'poisson'.",'squared_error'
,"hidden_layer_sizes hidden_layer_sizes: array-like of shape(n_layers - 2,), default=(100,)The ith element represents the number of neurons in the ithhidden layer.","(128, ...)"
,"activation activation: {'identity', 'logistic', 'tanh', 'relu'}, default='relu'Activation function for the hidden layer.- 'identity', no-op activation, useful to implement linear bottleneck, returns f(x) = x- 'logistic', the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).- 'tanh', the hyperbolic tan function, returns f(x) = tanh(x).- 'relu', the rectified linear unit function, returns f(x) = max(0, x)",'relu'
,"solver solver: {'lbfgs', 'sgd', 'adam'}, default='adam'The solver for weight optimization.- 'lbfgs' is an optimizer in the family of quasi-Newton methods.- 'sgd' refers to stochastic gradient descent.- 'adam' refers to a stochastic gradient-based optimizer proposed by Kingma, Diederik, and Jimmy BaFor a comparison between Adam optimizer and SGD, see:ref:`sphx_glr_auto_examples_neural_networks_plot_mlp_training_curves.py`.Note: The default solver 'adam' works pretty well on relativelylarge datasets (with thousands of training samples or more) in terms ofboth training time and validation score.For small datasets, however, 'lbfgs' can converge faster and performbetter.",'adam'
,"alpha alpha: float, default=0.0001Strength of the L2 regularization term. The L2 regularization termis divided by the sample size when added to the loss.",0.0001
,"batch_size batch_size: int, default='auto'Size of minibatches for stochastic optimizers.If the solver is 'lbfgs', the regressor will not use minibatch.When set to ""auto"", `batch_size=min(200, n_samples)`.",'auto'
,"learning_rate learning_rate: {'constant', 'invscaling', 'adaptive'}, default='constant'Learning rate schedule for weight updates.- 'constant' is a constant learning rate given by 'learning_rate_init'.- 'invscaling' gradually decreases the learning rate ``learning_rate_`` at each time step 't' using an inverse scaling exponent of 'power_t'. effective_learning_rate = learning_rate_init / pow(t, power_t)- 'adaptive' keeps the learning rate constant to 'learning_rate_init' as long as training loss keeps decreasing. Each time two consecutive epochs fail to decrease training loss by at least tol, or fail to increase validation score by at least tol if 'early_stopping' is on, the current learning rate is divided by 5.Only used when solver='sgd'.",'constant'
,"learning_rate_init learning_rate_init: float, default=0.001The initial learning rate used. It controls the step-sizein updating the weights. Only used when solver='sgd' or 'adam'.",0.001
,"power_t power_t: float, default=0.5The exponent for inverse scaling learning rate.It is used in updating effective learning rate when the learning_rateis set to 'invscaling'. Only used when solver='sgd'.",0.5
,"max_iter max_iter: int, default=200Maximum number of iterations. The solver iterates until convergence(determined by 'tol') or this number of iterations. For stochasticsolvers ('sgd', 'adam'), note that this determines the number of epochs(how many times each data point will be used), not the number ofgradient steps.",100
,"shuffle shuffle: bool, default=TrueWhether to shuffle samples in each iteration. Only used whensolver='sgd' or 'adam'.",True


În această etapă a fost construit un model de tip Neural Network mai complex, cu două straturi ascunse (128 și 64 neuroni), pentru a evalua dacă o arhitectură mai adâncă poate îmbunătăți performanța față de modelul de bază. Creșterea numărului de neuroni și a profunzimii rețelei oferă o capacitate mai mare de a învăța relații neliniare complexe din date. Acest model este utilizat exclusiv în scop experimental, pentru a compara efectul complexității asupra performanței de generalizare.


6. Evaluare Model V2

In [8]:
y_pred_v2 = nn_v2.predict(X_test)

mae_v2 = mean_absolute_error(y_test, y_pred_v2)
mse_v2 = mean_squared_error(y_test, y_pred_v2)
rmse_v2 = np.sqrt(mse_v2)
r2_v2 = r2_score(y_test, y_pred_v2)

mae_v2, mse_v2, rmse_v2, r2_v2


(7.890373730267951,
 127.01766772373671,
 np.float64(11.27021152080726),
 0.16959852146319987)

Modelul Neural Network mai complex a fost evaluat pe setul de test folosind aceleași metrici ca și modelul de bază: MAE, MSE, RMSE și R². Compararea acestor valori cu cele obținute de baseline permite o analiză obiectivă a performanței. Rezultatele arată că, în ciuda complexității crescute, modelul experimental obține erori mai mari și un scor R² mai scăzut, indicând o capacitate mai redusă de generalizare pe date nevăzute.


7. Tabel FINAL cu experimente

In [9]:
results = pd.DataFrame({
    "Model": ["Baseline NN", "NN deeper"],
    "MAE": [mae, mae_v2],
    "MSE": [mse, mse_v2],
    "RMSE": [rmse, rmse_v2],
    "R2": [r2, r2_v2]
})

results


,Model,MAE,MSE,RMSE,R2
0,Baseline NN,7.630590,119.257502,10.920508,0.220332
1,NN deeper,7.890374,127.017668,11.270212,0.169599


Tabelul de mai sus sintetizează rezultatele obținute pentru toate experimentele realizate. Compararea modelelor a fost realizată în principal pe baza metricii RMSE, deoarece aceasta este exprimată în aceeași unitate ca variabila țintă (ore pe săptămână) și penalizează mai sever erorile mari. Pe baza valorilor obținute, modelul de bază Neural Network a fost selectat ca model final, întrucât prezintă performanțe superioare și o stabilitate mai bună comparativ cu modelul mai complex.


Concluzie – Task 5: Census Modeling using Neural Network Regression

În cadrul acestui task a fost aplicată o abordare de tip regresie cu rețele neuronale asupra setului de date Census, având ca obiectiv predicția numărului de ore lucrate pe săptămână. Analiza a fost realizată utilizând datele preprocesate obținute în Task 1, asigurând consistența procesului și evitarea scurgerii de informație (data leakage). Inițial, a fost construit un model de bază (baseline) de tip Neural Network, care a servit drept punct de referință pentru evaluarea performanței.

Rezultatele obținute de modelul baseline indică o capacitate moderată de predicție, cu valori acceptabile ale erorilor și un scor R² pozitiv, ceea ce demonstrează că modelul reușește să explice o parte din variabilitatea variabilei țintă. Ulterior, a fost testată o arhitectură de rețea neuronală mai complexă, cu mai multe straturi ascunse, în scop experimental. Cu toate acestea, modelul mai complex nu a înregistrat îmbunătățiri ale performanței, prezentând valori mai mari ale RMSE și un scor R² mai scăzut comparativ cu modelul de bază.

Pe baza analizei comparative, modelul baseline de tip Neural Network a fost selectat ca model final, deoarece oferă o performanță mai bună și o capacitate de generalizare superioară pe datele de test. Aceste rezultate evidențiază faptul că o creștere a complexității modelului nu garantează automat o performanță mai bună și subliniază importanța evaluării riguroase a modelelor în problemele de regresie.
